<a href="https://colab.research.google.com/github/NAshwinKumar/credit_risk_assessment/blob/master/HGANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder 
from collections import defaultdict

In [0]:
data = pd.read_csv('http://invidio.drl.pl/files/german_credit.csv')
# data = pd.read_csv('https://raw.githubusercontent.com/vibhor98/German-Credit-Dataset/master/german_credit_data.csv')
data.rename(columns={'default' : 'class'},inplace=True)

In [0]:
data

,class,account_check_status,duration_in_month,credit_history,purpose,credit_amount,savings,present_emp_since,installment_as_income_perc,personal_status_sex,other_debtors,present_res_since,property,age,other_installment_plans,housing,credits_this_bank,job,people_under_maintenance,telephone,foreign_worker
0,0,< 0 DM,6,critical account/ other credits existing (not ...,domestic appliances,1169,unknown/ no savings account,.. >= 7 years,4,male : single,none,4,real estate,67,none,own,2,skilled employee / official,1,"yes, registered under the customers name",yes
1,1,0 <= ... < 200 DM,48,existing credits paid back duly till now,domestic appliances,5951,... < 100 DM,1 <= ... < 4 years,2,female : divorced/separated/married,none,2,real estate,22,none,own,1,skilled employee / official,1,none,yes
2,0,no checking account,12,critical account/ other credits existing (not ...,(vacation - does not exist?),2096,... < 100 DM,4 <= ... < 7 years,2,male : single,none,3,real estate,49,none,own,1,unskilled - resident,2,none,yes
3,0,< 0 DM,42,existing credits paid back duly till now,radio/television,7882,... < 100 DM,4 <= ... < 7 years,2,male : single,guarantor,4,if not A121 : building society savings agreeme...,45,none,for free,1,skilled employee / official,2,none,yes
4,1,< 0 DM,24,delay in paying off in the past,car (new),4870,... < 100 DM,1 <= ... < 4 years,3,male : single,none,4,unknown / no property,53,none,for free,2,skilled employee / official,2,none,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,no checking account,12,existing credits paid back duly till now,radio/television,1736,... < 100 DM,4 <= ... < 7 years,3,female : divorced/separated/married,none,4,real estate,31,none,own,1,unskilled - resident,1,none,yes
996,0,< 0 DM,30,existing credits paid back duly till now,car (used),3857,... < 100 DM,1 <= ... < 4 years,4,male : divorced/separated,none,4,if not A121 : building society savings agreeme...,40,none,own,1,management/ self-employed/ highly qualified em...,1,"yes, registered under the customers name",yes
997,0,no checking account,12,existing credits paid back duly till now,domestic appliances,804,... < 100 DM,.. >= 7 years,4,male : single,none,4,"if not A121/A122 : car or other, not in attrib...",38,none,own,1,skilled employee / official,1,none,yes
998,1,< 0 DM,45,existing credits paid back duly till now,domestic appliances,1845,... < 100 DM,1 <= ... < 4 years,4,male : single,none,4,unknown / no property,23,none,for free,1,skilled employee / official,1,"yes, registered under the customers name",yes


# Pre-processing

In [0]:
print ('\033[1m' + "Number of Rows     : " +'\033[0m', data.shape[0])
print ('\033[1m' + "\nNumber of Columns  : " +'\033[0m', data.shape[1])
print ('\033[1m' + "\nMissing values     :  "+'\033[0m', data.isnull().values.any())
print('\033[1m' + "\nColumn Names\t\t\tUnique Values  \n"+'\033[0m', data.nunique(),sep="")
print('\033[1m' + "\nColumn Names\t\t\tData Types  \n"+'\033[0m', data.dtypes,sep="")

Number of Rows     :  1000

Number of Columns  :  21

Missing values     :   False

Column Names			Unique Values  
class                           2
account_check_status            4
duration_in_month              33
credit_history                  5
purpose                        10
credit_amount                 921
savings                         5
present_emp_since               5
installment_as_income_perc      4
personal_status_sex             4
other_debtors                   3
present_res_since               4
property                        4
age                            53
other_installment_plans         3
housing                         3
credits_this_bank               4
job                             4
people_under_maintenance        2
telephone                       2
foreign_worker                  2
dtype: int64

Column Names			Data Types  
class                          int64
account_check_status          object
duration_in_month              int64
credit_history    

## Min-max normalisation

In [0]:
#column names of numeric variables 
numeric_variables = data.select_dtypes(exclude='object').drop(['class'], axis=1).columns #['creditamount', 'duration', 'installmentrate', 'residencesince', 'age', 'existingcredits', 'peopleliable', 'classification']

numeric_data = pd.DataFrame(StandardScaler().fit_transform(data[numeric_variables]))

## Encoding Categorical columns

In [0]:
#column names of ategorical variables 
categorical_variables = data.select_dtypes(include='object').columns  #['existingchecking', 'credithistory', 'purpose', 'savings', 'employmentsince', 'statussex', 'otherdebtors', 'property', 'otherinstallmentplans', 'housing', 'job', 'telephone', 'foreignworker']

# Encoding the variable
# d = defaultdict(LabelEncoder)

# lecatdata = data[categorical_variables].apply(lambda x: d[x.name].fit_transform(x))

# # print transformations
# for x in range(len(categorical_variables)):
#     print(categorical_variables[x],": ", data[categorical_variables[x]].unique())
#     print(categorical_variables[x],": ", lecatdata[categorical_variables[x]].unique())

#One hot encoding, create a column for every category of every categorical variable
categorical_data = pd.get_dummies(data[categorical_variables])

In [0]:
data_preprocessed = pd.concat([numeric_data, categorical_data, data['class']], axis = 1)
print(data_preprocessed.shape)

(1000, 62)
